In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Charger les données générées à l'étape 1
df = pd.read_csv("../data/generated/cafe_data_2024.csv")
df['date'] = pd.to_datetime(df['date'])  # Conversion obligatoire pour analyse temporelle

# 2. Diagnostic rapide des données
print(" SHAPE :", df.shape)           # (365, 4) → 365 jours, 4 colonnes
print("\n TYPES :\n", df.dtypes)      # Vérifier que 'date' est datetime64
print("\n VALEURS MANQUANTES :\n", df.isnull().sum())  # Doit être 0 partout
print("\n STATISTIQUES :\n", df.describe().round(2))  # Min/max/moyenne/écart-type

# 3. Détection d'anomalies métier
print("\n  ANOMALIES :")
print(f"  • Ventes négatives : {(df['ventes_total'] < 0).sum()}")
print(f"  • Clients = 0      : {(df['nb_clients'] == 0).sum()}")
print(f"  • Ventes > 200€    : {(df['ventes_total'] > 200).sum()}")  # Seuil métier à ajuster

# 4. Impact weekend (KPI décisionnel)
weekend_avg = df[df['est_weekend']]['ventes_total'].mean()
weekday_avg = df[~df['est_weekend']]['ventes_total'].mean()
boost_pct = ((weekend_avg / weekday_avg) - 1) * 100
print(f"\n KPI WEEKEND :")
print(f"  • Moyenne weekend : {weekend_avg:.2f}€")
print(f"  • Moyenne semaine : {weekday_avg:.2f}€")
print(f"  • Boost           : +{boost_pct:.1f}%")

# 5. Visualisation saisonnalité + weekend
plt.figure(figsize=(14, 5))
sns.lineplot(data=df, x='date', y='ventes_total', hue='est_weekend', palette={True: 'red', False: 'blue'})
plt.axhline(y=df['ventes_total'].mean(), color='green', linestyle='--', label='Moyenne annuelle')
plt.title("Ventes 2024 — Saisonnalité + Effet Weekend", fontsize=14, fontweight='bold')
plt.ylabel("Ventes (€)")
plt.xlabel("Date")
plt.legend(title='Weekend')
plt.tight_layout()
plt.savefig("../outputs/figures/eda_ventes.png", dpi=150, bbox_inches='tight')
plt.show()

# 6. Distribution des ventes (détecter les outliers)
plt.figure(figsize=(8, 4))
sns.histplot(df['ventes_total'], bins=30, kde=True)
plt.title("Distribution des ventes journalières")
plt.xlabel("Ventes (€)")²
plt.tight_layout()
plt.savefig("../outputs/figures/eda_distribution.png", dpi=150)
plt.show()

: 